# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data=pd.read_csv("G:\study\\bse\\PIAIC\\q2\\archive (1)\\CarPrice_Assignment.csv")

In [3]:
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
car_ID              205 non-null int64
symboling           205 non-null int64
CarName             205 non-null object
fueltype            205 non-null object
aspiration          205 non-null object
doornumber          205 non-null object
carbody             205 non-null object
drivewheel          205 non-null object
enginelocation      205 non-null object
wheelbase           205 non-null float64
carlength           205 non-null float64
carwidth            205 non-null float64
carheight           205 non-null float64
curbweight          205 non-null int64
enginetype          205 non-null object
cylindernumber      205 non-null object
enginesize          205 non-null int64
fuelsystem          205 non-null object
boreratio           205 non-null float64
stroke              205 non-null float64
compressionratio    205 non-null float64
horsepower          205 non-null int64
peakrpm      

In [5]:
x=data.drop(["price","car_ID","CarName"],axis=1).copy()
y=data["price"].copy()

In [6]:
{column: list(x[column].unique()) for column in x.columns if x.dtypes[column]=='object'}

{'fueltype': ['gas', 'diesel'],
 'aspiration': ['std', 'turbo'],
 'doornumber': ['two', 'four'],
 'carbody': ['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
 'drivewheel': ['rwd', 'fwd', '4wd'],
 'enginelocation': ['front', 'rear'],
 'enginetype': ['dohc', 'ohcv', 'ohc', 'l', 'rotor', 'ohcf', 'dohcv'],
 'cylindernumber': ['four', 'six', 'five', 'three', 'twelve', 'two', 'eight'],
 'fuelsystem': ['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi']}

In [7]:
x["doornumber"]=x["doornumber"].apply(lambda x: 0 if x == 'two' else 1)

In [8]:
numeric_ordering ={'two':2,'three':3,'four':4,'five':5,'six':6,'eight':8,'twelve':12}

In [9]:
x["cylindernumber"] =x["cylindernumber"].replace(numeric_ordering)

In [10]:
def binary_encode(x, columns, positive_values):
    x =x.copy()
    for column, positive_value in zip(columns, positive_values):
        x[column] = x[column].apply(lambda x: 1 if x == positive_value else 0)
    return x

In [11]:
binary_features=['fueltype','aspiration','enginelocation',]
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [12]:
binary_features = ['fueltype','aspiration','enginelocation']
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [13]:
def onehot_encode(x, columns, prefixes):
    x = x.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(x[column], prefix=prefix)
        x=pd.concat([x, dummies], axis=1)
        x=x.drop(column, axis=1)
    return x

In [14]:
nominal_features=['carbody','drivewheel','enginetype','fuelsystem']
prefixes=['MK','BS','DW','ET','FS']
x=onehot_encode(x,columns=nominal_features,prefixes=prefixes)

In [15]:
x

,symboling,fueltype,aspiration,doornumber,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,...,DW_ohcv,DW_rotor,ET_1bbl,ET_2bbl,ET_4bbl,ET_idi,ET_mfi,ET_mpfi,ET_spdi,ET_spfi
0,3,0,0,0,0,88.6,168.8,64.1,48.8,2548,...,0,0,0,0,0,0,0,1,0,0
1,3,0,0,0,0,88.6,168.8,64.1,48.8,2548,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,94.5,171.2,65.5,52.4,2823,...,1,0,0,0,0,0,0,1,0,0
3,2,0,0,1,0,99.8,176.6,66.2,54.3,2337,...,0,0,0,0,0,0,0,1,0,0
4,2,0,0,1,0,99.4,176.6,66.4,54.3,2824,...,0,0,0,0,0,0,0,1,0,0
5,2,0,0,0,0,99.8,177.3,66.3,53.1,2507,...,0,0,0,0,0,0,0,1,0,0
6,1,0,0,1,0,105.8,192.7,71.4,55.7,2844,...,0,0,0,0,0,0,0,1,0,0
7,1,0,0,1,0,105.8,192.7,71.4,55.7,2954,...,0,0,0,0,0,0,0,1,0,0
8,1,0,0,1,0,105.8,192.7,71.4,55.9,3086,...,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,99.5,178.2,67.9,52.0,3053,...,0,0,0,0,0,0,0,1,0,0


In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=123)

In [17]:
scaler = StandardScaler()
scaler.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [18]:
x_train =pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)

In [19]:
part_x_train,x_val,part_y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=2)

In [24]:
#using relu

In [27]:
model = Sequential()
#input layer
model.add(Dense(10, input_dim=42, kernel_initializer='normal', activation='relu'))
#hidden layers
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [28]:
history=model.fit(part_x_train,part_y_train,epochs=100,validation_data=(x_val,y_val),batch_size=512)

Train on 114 samples, validate on 29 samples
Epoch 1/100
114/114 [==============================] - 1s 9ms/sample - loss: 263507504.0000 - mse: 263507504.0000 - mae: 14128.1660 - val_loss: 334189984.0000 - val_mse: 334189984.0000 - val_mae: 14966.6455
Epoch 2/100
114/114 [==============================] - 0s 161us/sample - loss: 263507408.0000 - mse: 263507408.0000 - mae: 14128.1641 - val_loss: 334189792.0000 - val_mse: 334189792.0000 - val_mae: 14966.6426
Epoch 3/100
114/114 [==============================] - 0s 140us/sample - loss: 263507344.0000 - mse: 263507344.0000 - mae: 14128.1611 - val_loss: 334189568.0000 - val_mse: 334189568.0000 - val_mae: 14966.6367
Epoch 4/100
114/114 [==============================] - 0s 118us/sample - loss: 263507184.0000 - mse: 263507184.0000 - mae: 14128.1592 - val_loss: 334189312.0000 - val_mse: 334189312.0000 - val_mae: 14966.6338
Epoch 5/100
114/114 [==============================] - 0s 194us/sample - loss: 263507072.0000 - mse: 263507072.0000 - mae

Epoch 40/100
114/114 [==============================] - 0s 109us/sample - loss: 263495680.0000 - mse: 263495680.0000 - mae: 14127.8145 - val_loss: 334172832.0000 - val_mse: 334172832.0000 - val_mae: 14966.2275
Epoch 41/100
114/114 [==============================] - 0s 114us/sample - loss: 263495056.0000 - mse: 263495056.0000 - mae: 14127.7959 - val_loss: 334172096.0000 - val_mse: 334172096.0000 - val_mae: 14966.2070
Epoch 42/100
114/114 [==============================] - 0s 118us/sample - loss: 263494400.0000 - mse: 263494400.0000 - mae: 14127.7754 - val_loss: 334171264.0000 - val_mse: 334171264.0000 - val_mae: 14966.1846
Epoch 43/100
114/114 [==============================] - 0s 111us/sample - loss: 263493776.0000 - mse: 263493776.0000 - mae: 14127.7539 - val_loss: 334170496.0000 - val_mse: 334170496.0000 - val_mae: 14966.1641
Epoch 44/100
114/114 [==============================] - 0s 108us/sample - loss: 263493136.0000 - mse: 263493136.0000 - mae: 14127.7334 - val_loss: 334169696.000

114/114 [==============================] - 0s 106us/sample - loss: 263451168.0000 - mse: 263451168.0000 - mae: 14126.2627 - val_loss: 334117760.0000 - val_mse: 334117760.0000 - val_mae: 14964.5752
Epoch 80/100
114/114 [==============================] - 0s 111us/sample - loss: 263449136.0000 - mse: 263449136.0000 - mae: 14126.1895 - val_loss: 334115360.0000 - val_mse: 334115360.0000 - val_mae: 14964.4980
Epoch 81/100
114/114 [==============================] - 0s 99us/sample - loss: 263447072.0000 - mse: 263447072.0000 - mae: 14126.1162 - val_loss: 334112768.0000 - val_mse: 334112768.0000 - val_mae: 14964.4189
Epoch 82/100
114/114 [==============================] - 0s 163us/sample - loss: 263444928.0000 - mse: 263444928.0000 - mae: 14126.0381 - val_loss: 334110176.0000 - val_mse: 334110176.0000 - val_mae: 14964.3369
Epoch 83/100
114/114 [==============================] - 0s 159us/sample - loss: 263442704.0000 - mse: 263442704.0000 - mae: 14125.9590 - val_loss: 334107552.0000 - val_mse: 3

In [30]:
model.evaluate(x_test,y_test)

62/62 [==============================] - 0s 138us/sample - loss: 151906252.3871 - mse: 151906256.0000 - mae: 10916.3711


[151906252.38709676, 151906260.0, 10916.371]

In [31]:
model.predict(x_test).sum()

266.0314

In [32]:
#using tanh

In [33]:
model = Sequential()
#input layer
model.add(Dense(10, input_dim=42, kernel_initializer='normal', activation='relu'))
#hidden layers
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [34]:
history=model.fit(part_x_train,part_y_train,epochs=100,validation_data=(x_val,y_val),batch_size=512)

Train on 114 samples, validate on 29 samples
Epoch 1/100
114/114 [==============================] - 1s 7ms/sample - loss: 263509536.0000 - mse: 263509536.0000 - mae: 14128.2529 - val_loss: 334191936.0000 - val_mse: 334191936.0000 - val_mae: 14966.7373
Epoch 2/100
114/114 [==============================] - 0s 116us/sample - loss: 263509232.0000 - mse: 263509232.0000 - mae: 14128.2393 - val_loss: 334191616.0000 - val_mse: 334191616.0000 - val_mae: 14966.7256
Epoch 3/100
114/114 [==============================] - 0s 128us/sample - loss: 263508896.0000 - mse: 263508896.0000 - mae: 14128.2266 - val_loss: 334191328.0000 - val_mse: 334191328.0000 - val_mae: 14966.7129
Epoch 4/100
114/114 [==============================] - 0s 111us/sample - loss: 263508576.0000 - mse: 263508576.0000 - mae: 14128.2139 - val_loss: 334190976.0000 - val_mse: 334190976.0000 - val_mae: 14966.7012
Epoch 5/100
114/114 [==============================] - 0s 177us/sample - loss: 263508304.0000 - mse: 263508304.0000 - mae

Epoch 40/100
114/114 [==============================] - 0s 114us/sample - loss: 263499616.0000 - mse: 263499616.0000 - mae: 14127.9170 - val_loss: 334180672.0000 - val_mse: 334180672.0000 - val_mae: 14966.3896
Epoch 41/100
114/114 [==============================] - 0s 133us/sample - loss: 263499280.0000 - mse: 263499280.0000 - mae: 14127.9072 - val_loss: 334180256.0000 - val_mse: 334180256.0000 - val_mae: 14966.3789
Epoch 42/100
114/114 [==============================] - 0s 107us/sample - loss: 263498928.0000 - mse: 263498928.0000 - mae: 14127.8965 - val_loss: 334179808.0000 - val_mse: 334179808.0000 - val_mae: 14966.3672
Epoch 43/100
114/114 [==============================] - 0s 105us/sample - loss: 263498608.0000 - mse: 263498608.0000 - mae: 14127.8848 - val_loss: 334179456.0000 - val_mse: 334179456.0000 - val_mae: 14966.3564
Epoch 44/100
114/114 [==============================] - 0s 111us/sample - loss: 263498256.0000 - mse: 263498256.0000 - mae: 14127.8740 - val_loss: 334179008.000

Epoch 80/100
114/114 [==============================] - 0s 117us/sample - loss: 263487104.0000 - mse: 263487104.0000 - mae: 14127.4834 - val_loss: 334166976.0000 - val_mse: 334166976.0000 - val_mae: 14965.9385
Epoch 81/100
114/114 [==============================] - 0s 129us/sample - loss: 263486880.0000 - mse: 263486880.0000 - mae: 14127.4746 - val_loss: 334166816.0000 - val_mse: 334166816.0000 - val_mae: 14965.9307
Epoch 82/100
114/114 [==============================] - 0s 122us/sample - loss: 263486720.0000 - mse: 263486720.0000 - mae: 14127.4668 - val_loss: 334166592.0000 - val_mse: 334166592.0000 - val_mae: 14965.9238
Epoch 83/100
114/114 [==============================] - 0s 132us/sample - loss: 263486464.0000 - mse: 263486464.0000 - mae: 14127.4609 - val_loss: 334166368.0000 - val_mse: 334166368.0000 - val_mae: 14965.9160
Epoch 84/100
114/114 [==============================] - 0s 113us/sample - loss: 263486304.0000 - mse: 263486304.0000 - mae: 14127.4521 - val_loss: 334166144.000

In [35]:
model.evaluate(x_test,y_test)

62/62 [==============================] - 0s 113us/sample - loss: 151979094.1935 - mse: 151979088.0000 - mae: 10919.8525


[151979094.19354838, 151979090.0, 10919.853]

In [36]:
model.predict(x_test).sum()

50.15152